In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import tkinter as tk
from tkinter import messagebox


In [3]:
# Charger le corpus
corpus_path = r"D:\\etude\\M2\\RI\\nfcorpus\\corpus.txt"
with open(corpus_path, 'r', encoding='utf-8') as file:
    corpus = file.readlines()


In [4]:
# Calculer TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)


In [5]:
# Fonction Rocchio
def rocchio_update(Q0, relevant_docs, non_relevant_docs, alpha=1, beta=0.75, gamma=0.15):
    centroid_relevant = np.mean(relevant_docs, axis=0) if relevant_docs.size > 0 else np.zeros_like(Q0)
    centroid_non_relevant = np.mean(non_relevant_docs, axis=0) if non_relevant_docs.size > 0 else np.zeros_like(Q0)
    Q1 = alpha * Q0 + beta * centroid_relevant - gamma * centroid_non_relevant
    return Q1



In [6]:
# Interface pour sélection des documents
class RocchioInterface:
    def __init__(self, master):
        self.master = master
        self.query_vector = None
        self.selected_relevant = []
        self.selected_non_relevant = []
        self.displayed_docs = []
        self.similarities = []

        # Interface utilisateur
        self.master.title("Recherche et Sélection de Documents")
        
        # Champ pour saisir la requête
        self.query_label = tk.Label(master, text="Entrez votre requête :")
        self.query_label.pack()
        self.query_entry = tk.Entry(master, width=50)
        self.query_entry.pack()
        self.search_button = tk.Button(master, text="Rechercher", command=self.perform_search)
        self.search_button.pack()

        # Instructions pour la sélection des documents
        self.label = tk.Label(master, text="Sélectionnez les documents pertinents parmi les 15 premiers :")
        self.label.pack()
        
        # Liste des documents
        self.listbox = tk.Listbox(master, selectmode=tk.MULTIPLE, width=100, height=20)
        self.listbox.pack()

        # Bouton pour soumettre les documents pertinents
        self.submit_button = tk.Button(master, text="Soumettre", command=self.submit)
        self.submit_button.pack()

    def perform_search(self):
        query = self.query_entry.get().strip()
        if not query:
            messagebox.showerror("Erreur", "Veuillez entrer une requête valide.")
            return

        # Calculer le vecteur de la requête et les similarités
        self.query_vector = vectorizer.transform([query]).toarray()
        self.similarities = cosine_similarity(self.query_vector, tfidf_matrix).flatten()

        # Afficher les 15 premiers résultats
        self.update_display(limit=15)

    def update_display(self, limit=15):
      if self.similarities is None or len(self.similarities) == 0:
          return

      top_indices = np.argsort(self.similarities)[-limit:][::-1]  # Les `limit` plus pertinents
      self.displayed_docs = [(i, corpus[i]) for i in top_indices]

      # Mettre à jour la liste affichée
      self.listbox.delete(0, tk.END)
      for idx, doc in self.displayed_docs:
          self.listbox.insert(tk.END, f"Doc {idx}: {doc[:100]}...")
    
    def calculate_precision_recall(self, relevant_indices, retrieved_indices):
      # Documents pertinents réellement récupérés
      relevant_retrieved = len(set(relevant_indices) & set(retrieved_indices))
      
      # Calcul de la précision
      precision = relevant_retrieved / len(retrieved_indices) if len(retrieved_indices) > 0 else 0

      # Calcul du rappel
      recall = relevant_retrieved / len(relevant_indices) if len(relevant_indices) > 0 else 0
      
      return precision, recall

    def submit(self):
        if not self.query_vector.any():
            messagebox.showerror("Erreur", "Veuillez d'abord effectuer une recherche.")
            return

        # Obtenir les sélections
        selected_indices = self.listbox.curselection()
        relevant_indices = [self.displayed_docs[i][0] for i in selected_indices]

        # Documents récupérés (indices affichés)
        retrieved_indices = [doc[0] for doc in self.displayed_docs]

        # Calculer la précision et le rappel
        precision, recall = self.calculate_precision_recall(relevant_indices, retrieved_indices)

        # Afficher les résultats
        messagebox.showinfo("Évaluation", f"Précision : {precision:.2f}\nRappel : {recall:.2f}")

        # Séparer les documents pertinents et non pertinents
        self.selected_relevant = tfidf_matrix[relevant_indices].toarray()
        self.selected_non_relevant = tfidf_matrix[[i for i, _ in self.displayed_docs if i not in relevant_indices]].toarray()

        # Mettre à jour la requête avec Rocchio (première itération)
        self.query_vector = rocchio_update(
            self.query_vector,
            self.selected_relevant,
            self.selected_non_relevant
        )

        # Recalculer les similarités avec la nouvelle requête (q1)
        self.similarities = cosine_similarity(self.query_vector, tfidf_matrix).flatten()

        # Appliquer Rocchio une deuxième fois pour affiner (q2)
        top_indices = np.argsort(self.similarities)[-20:][::-1]  # Sélectionner 20 documents
        refined_relevant = tfidf_matrix[top_indices].toarray()
        self.query_vector = rocchio_update(
            self.query_vector,
            refined_relevant,
            self.selected_non_relevant
        )

        # Mettre à jour les similarités et afficher les 20 documents suivants
        self.similarities = cosine_similarity(self.query_vector, tfidf_matrix).flatten()
        self.update_display(limit=20)

        # Informer l'utilisateur
        messagebox.showinfo("Mise à jour", "Requête mise à jour. Les 20 prochains documents pertinents sont affichés.")


In [7]:
# Lancer l'interface
if __name__ == "__main__":
    root = tk.Tk()
    app = RocchioInterface(root)
    root.mainloop()
